In [ ]:
!pip install transformers datasets evaluate rouge_score

In [ ]:
from huggingface_hub import notebook_login, login

login("TOKEN KEY")

In [ ]:
from datasets import load_dataset

news = load_dataset("multi_news", split="train")

In [ ]:
news = news.train_test_split(test_size=0.2)

In [ ]:
news["train"][0]

In [ ]:
from transformers import AutoTokenizer

checkpoint = "microsoft/prophetnet-large-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
prefix = "summarize: "

def preprocess_data(examples):
    # Prepend prefix to each report in examples
    documents = examples["article"]
    prepended_documents = [prefix + document for document in documents]

    # Tokenize the prepended documents
    tokenized_inputs = tokenizer(prepended_documents, max_length=1024, truncation=True)

    # Tokenize the summary labels
    summaries = examples["summary"]
    tokenized_labels = tokenizer(text_target=summaries, max_length=128, truncation=True)

    # Assign the tokenized labels to the "labels" key in the tokenized_inputs dictionary
    tokenized_inputs["labels"] = tokenized_labels["input_ids"]
    
    return tokenized_inputs

In [ ]:
tokenized_news = news.map(preprocess_function, batched=True, num_proc=100)

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [ ]:
import evaluate

rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")

In [ ]:
import numpy as np
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu
from nltk.translate.meteor_score import meteor_score
from concurrent.futures import ThreadPoolExecutor

def compute_bleu_score(pred_ref_pair):
    pred, ref = pred_ref_pair
    smoothing = SmoothingFunction().method1 
    return sentence_bleu([ref], pred, smoothing_function=smoothing)

def metrics_evaluation(evaluation_predictions):
    predicted_outputs, associated_labels = evaluation_predictions
    decoded_predicted_outputs = tokenizer.batch_decode(predicted_outputs, skip_special_tokens=True)
    processed_labels = np.where(associated_labels != -100, associated_labels, tokenizer.pad_token_id)
    decoded_processed_labels = tokenizer.batch_decode(processed_labels, skip_special_tokens=True)

    # Use Rouge to compute scores for the decoded predictions and decoded labels
    computed_rouge_scores = rouge.compute(predictions=decoded_predicted_outputs, references=decoded_processed_labels, use_stemmer=True)

    return computed_rouge_scores

    # Compute BLEU scores
    with ThreadPoolExecutor() as executor:
        bleu_scores = list(executor.map(compute_bleu_score, zip(decoded_preds, decoded_labels)))
    result["bleu"] = np.mean(bleu_scores)

    # Compute average length of generated predictions
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
!apt install git-lfs

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="Prophetnet_multiNews_Model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=14,
    per_device_eval_batch_size=14,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_news["train"],
    eval_dataset=tokenized_news["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()